In [1]:
import tensorflow as tf
import numpy as np
import os
slim = tf.contrib.slim

#read data
def get_one_batch(batch_size,type):
    if type == 0: #train
        file_list = tf.gfile.Glob("train*.tfrecords")
    else:
        file_list = tf.gfile.Glob("test*.tfrecords")

    reader = tf.TFRecordReader()
    filename_queue = tf.train.string_input_producer(file_list,num_epochs=None,shuffle=True)
    _,se = reader.read(filename_queue)
    
    if type==0:
        batch = tf.train.shuffle_batch([se],batch_size,capacity=batch_size,min_after_dequeue=batch_size//2)
    else:
        batch = tf.train.batch([se],batch_size,capacity=batch_size)
        
    features = tf.parse_example(batch,features={
        'image':tf.FixedLenFeature([],tf.string),
        'label':tf.FixedLenFeature([136],tf.float32)
    })
    
    batch_im = features['image']
    batch_label = features['label']
    batch_im = tf.decode_raw(batch_im,tf.uint8)
    batch_im = tf.cast(tf.reshape(batch_im,(batch_size,128,128,3)),tf.float32)
    return batch_im,batch_label
        
#net
def senet_blob(net,in_dim,out_dim,stride):#in_dim输入特征图通道数，Out_dim输出，stride:是否下采样
    bk = net
    net = slim.conv2d(net,in_dim//4,[1,1])
    net = slim.conv2d(net,in_dim//4,[3,3])
    net = slim.conv2d(net,out_dim,[1,1])
    if stride>1:
        net = slim.avg_pool2d(net,[stride*2-1,stride*2-1],stride = stride, padding="SAME")
        bk = slim.avg_pool2d(bk,[stride*2-1,stride*2-1],stride = stride, padding="SAME")
    if in_dim != out_dim:
        bk = slim.conv2d(bk,out_dim,[1,1])
    
    squeeze = tf.reduce_mean(net,axis=[1,2])
    ex = slim.fully_connected(squeeze,out_dim//16)
    ex = tf.nn.relu(ex)
    ex = slim.fully_connected(squeeze,out_dim)
    ex = tf.nn.sigmoid(ex)
    net = net * tf.reshape(ex,[-1,1,1,out_dim])#attention部分
    net = bk + net#res部分
    return net
    
def SENet(input_x,is_training=True,keep_prob=0.8):
    bn_param = {
        'is_training':is_training,
        'decay':0.997,
        'epsilon':1e-5,
        'scale':True,
        'updates_collections':tf.GraphKeys.UPDATE_OPS
    }
    with slim.arg_scope([slim.conv2d],
                        weights_regularizer=slim.l2_regularizer(0.00001),
                        weights_initializer=slim.variance_scaling_initializer(),
                        activation_fn=tf.nn.relu,
                        normalizer_fn=slim.batch_norm,
                        normalizer_params=bn_param):
        with slim.arg_scope([slim.batch_norm],**bn_param):
            net = slim.conv2d(input_x,32,[3,3])
            net = slim.avg_pool2d(net,[3,3],stride=2,padding="SAME")
            net = senet_blob(net,32,64,2)
            net = senet_blob(net,64,128,2)
            net = senet_blob(net,128,256,2)
            net = senet_blob(net,256,512,2)
            net = tf.reduce_mean(net,axis=[1,2])
            net = slim.fully_connected(net,1024)
            net = tf.nn.relu(net)
            net = tf.nn.dropout(net,keep_prob=keep_prob)
            net = slim.fully_connected(net,136)
        
    return net


input_x = tf.placeholder(tf.float32,shape = [1,128,128,3])
logits = SENet(input_x,is_training=False,keep_prob=1.0)
print('aaaaaaaaaaaaa',logits)

saver = tf.train.Saver(tf.global_variables())

#session
with tf.Session() as sess:
    coord = tf.train.Coordinator()#线程读取数据
    tf.train.start_queue_runners(sess=sess,coord=coord)
    init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
    sess.run(init_op)
    #ckpt = tf.train.get_checkpoint_state(model_dir)
    ckpt = tf.train.get_checkpoint_state("models")
    saver.restore(sess,os.path.join("models", "senet.ckpt22500"))
    output_graph_def = tf.graph_util.convert_variables_to_constants(sess,
                                                                    sess.graph.as_graph_def(),
                                                                    ['fully_connected_9/Relu'])#想要保存的结点输出，
                                                                                            #此时为最后一层输出的名字
    with tf.gfile.FastGFile('landmark.pb','wb') as f:
        f.write(output_graph_def.SerializeToString())
        f.close()
        

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
aaaaaaaaaaaaa Tensor("fully_connected_9/Relu:0", shape=(1, 136), dtype=float32)
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models\senet.ckpt22500
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 97 variables.
INFO:tensorflow:Converted 97 variables to const ops.
Instructions f